In [1]:
!pip install --upgrade --quiet pip 
!pip install -I --upgrade --quiet setuptools
!pip install --upgrade --quiet tensorflow edward pymongo
!pip install --quiet lightgbm

In [2]:
% matplotlib inline

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import fetch.fetch as fetch
from edward.models import Normal
from modelling.models import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *

In [5]:
df = fetch.create_df(fetch.get_labelled_articles("209.177.92.45:80"))

In [6]:
v = vectorizer(df)
tfidf = create_tfidf(v, df)
word_count = create_word_count(df)

In [7]:
model = MultinomialNB(class_prior=[.9,.1])
# errors = add_predictions(model, df, cv = 10
# ans = [1 if l == 'accepted' else 0 for l in df.label]

prediction_probs = cross_val_predict(model, tfidf, df.label, cv=10, method = 'predict_proba')
predictions = cross_val_predict(model, tfidf, df.label, cv=10, method = 'predict')
# errors.shape
preds = ['accepted' if p[0] >= 0.5 else 'rejected' for p in prediction_probs]
# np.sum(np.round(preds) != np.array(ans))

print(np.sum(predictions != df.label))

print(roc_auc_score(df.label == 'accepted', predictions == 'accepted'))
print(precision_score(df.label == 'accepted', predictions == 'accepted'))
print(average_precision_score(df.label == 'accepted', [p[0] for p in prediction_probs]))

561
0.692434210526
0.700640341515
0.896046619616


In [70]:
df[df.label != 'accepted'].shape

(893, 10)

In [133]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(5, shuffle = True)

def evaluate(model, tfidf, df, method = 'predict_proba'):
    ans = [1 if l == 'accepted' else 0 for l in df.label]
    probs = cross_val_predict(model, tfidf, ans, cv=cv, method=method)
    predictions = cross_val_predict(model, tfidf, ans, cv=cv, method='predict')

    probs = probs if method == 'decision_function' else [p[1] for p in probs] 
                           
    print (average_precision_score(ans, probs), roc_auc_score(ans, probs, average = None))
    print (precision_score(ans, predictions), recall_score(ans, predictions), fbeta_score(ans, predictions, beta = 1.5))
    # precision, recall, _ = precision_recall_curve(ans, guess)
    fpr, tpr, _ = roc_curve(ans, probs)
    plt.step(fpr, tpr)
    # plt.step(precision, recall)
    return probs, ans, predictions

In [23]:
from sklearn.model_selection import GridSearchCV



In [115]:
import lightgbm
from sklearn.metrics import fbeta_score, make_scorer

model = lightgbm.LGBMClassifier(n_estimators = 30)
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
grid = GridSearchCV(lightgbm.LGBMClassifier(max_depth = 2, n_estimators = 100, reg_lambda = 0.4, reg_alpha = 0.4), parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, df.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

   mean_fit_time  mean_score_time  mean_test_score  mean_train_score params  \
0       0.120874         0.014947         0.855022          0.933722     {}   

   rank_test_score  split0_test_score  split0_train_score  split1_test_score  \
0                1           0.763403            0.950115           0.907191   

   split1_train_score  split2_test_score  split2_train_score  std_fit_time  \
0            0.907645           0.894525            0.943406      0.008145   

   std_score_time  std_test_score  std_train_score  
0        0.005974        0.065012         0.018641  

In [141]:
df_tweets = df
df._id

                          _id                   added  cluster     label  \
0     ge:11721998312442412795 2016-11-06 06:19:17.960      NaN  accepted   
1      ge:1620759028784135434 2016-11-06 06:19:17.959      NaN  rejected   
2     ge:18239534028951138362 2016-11-06 06:19:17.959      NaN  rejected   
3     ge:12121811051108214999 2016-11-06 06:19:17.960      NaN  accepted   
4      ge:8991470599314538820 2016-11-06 06:19:17.960      NaN  rejected   
5     ge:12075882848350770687 2016-11-06 06:19:17.961      NaN  rejected   
6     ge:15428506610722352235 2016-11-06 06:19:17.961      NaN  accepted   
7      ge:5055609828518538266 2016-11-06 06:19:17.961      NaN  accepted   
8      ge:5637883152692138122 2016-11-06 06:19:17.961      NaN  accepted   
9      ge:7581265598718420349 2016-11-06 06:19:17.961      NaN  accepted   
10    ge:17574053986436593806 2016-11-06 06:19:17.961      NaN  rejected   
11     ge:8532986235004036463 2016-11-06 06:19:17.962      NaN  rejected   
12    ge:110

In [137]:
model = lightgbm.LGBMClassifier(max_depth = 2, n_estimators = 100, reg_lambda = 0.4, reg_alpha = 0.4)
_ = evaluate(model, word_count, df)

0.944428253614 0.932963716412
0.844908968307 0.95430312262 0.917741844611


In [140]:
model

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=255, max_depth=2, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=100, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0.4, reg_lambda=0.4, seed=0,
        silent=True, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)

In [135]:
from sklearn import svm

model = svm.LinearSVC()
g,a,p = evaluate(model, word_count, df, 'decision_function')

0.978473376403 0.975103886907
0.845755022683 0.993907083016 0.94307632442


In [ ]:
model = svm.OneClassSVM()
g,a,p = evaluate(model, tfidf, df, 'decision_function')

In [136]:
model = MultinomialNB(alpha = .001, class_prior = [0.45,0.55])
g, ans, preds = evaluate(model, word_count, df)

0.974158413959 0.968473163106
0.940221402214 0.970297029703 0.960840053374
